<a href="https://colab.research.google.com/github/Krobo224/Fake-News-Prediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FAKE NEWS PREDICTION**

In [2]:
#importing the libraries
import numpy as np
import pandas as pd
import re       #Regular Expression used to search text in document
from nltk.corpus import stopwords  #nltk:- Natural Language Toolkit
from nltk.stem.porter import PorterStemmer  # Stemming: Removes the prefix and suffix of a word and returns the original word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words('english')) # stopwords are words which dont add much value to data

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [31]:
#load data
news_data = pd.read_csv('/content/train.csv')
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [32]:
news_data.shape

(20800, 5)

In [33]:
# Counting the number of missing values
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [34]:
# As we have large dataset we will
# replace the null values with empty string
news_data = news_data.fillna('')

In [35]:
# mearging the features
news_data['content'] = news_data['author'] + ' ' + news_data['title'] + ' ' + news_data['text']
news_data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: content, Length: 20800, dtype: object

In [36]:
# Separating the data & label
X = news_data.drop(columns='label', axis=1)
Y = news_data['label']
X, Y

(          id                                              title  \
 0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
 1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
 2          2                  Why the Truth Might Get You Fired   
 3          3  15 Civilians Killed In Single US Airstrike Hav...   
 4          4  Iranian woman jailed for fictional unpublished...   
 ...      ...                                                ...   
 20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
 20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
 20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
 20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
 20799  20799                          What Keeps the F-35 Alive   
 
                                           author  \
 0                                  Darrell Lucus   
 1                                Daniel J. Flynn   
 2                     

In [13]:
X.shape, Y.shape

((20800, 5), (20800,))

Stemming Procedure

Stemming: Removing suffix or prefix of word and returning its original word

In [37]:
port_stem = PorterStemmer()

In [38]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-z]',' ',content) # this will remove all the symbols numbers, spaces except letters
  stemmed_content = stemmed_content.lower()           # converts to lower case
  stemmed_content = stemmed_content.split()           # splitted and converted to list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # returns original word except the stopwords
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [39]:
news_data['content'] = news_data['content'].apply(stemming)

In [40]:
news_data['content'].head()

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2    consortiumnew com truth might get fire truth m...
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object

In [41]:
X = news_data['content'].values
Y = news_data['label'].values
X, Y

(array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light

In [42]:
X.shape, Y.shape

((20800,), (20800,))

In [43]:
#Converting texual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)
print(X)

  (0, 110891)	0.04915770724915835
  (0, 110836)	0.019064436451004835
  (0, 109873)	0.04416489748909235
  (0, 109869)	0.09477377367763555
  (0, 109825)	0.037584418281075054
  (0, 109785)	0.011307434114810647
  (0, 109132)	0.01709233626132696
  (0, 108308)	0.017105726087083304
  (0, 108217)	0.012543079804529688
  (0, 108132)	0.02912605853670183
  (0, 108052)	0.012863161323367697
  (0, 107849)	0.011771571415539016
  (0, 106994)	0.025726881207843295
  (0, 106957)	0.03129631609205721
  (0, 105935)	0.021536230411571763
  (0, 104503)	0.06544474829832397
  (0, 103810)	0.03314878037992261
  (0, 103554)	0.01639762339961203
  (0, 102780)	0.03807145628518518
  (0, 102136)	0.011082267003706438
  (0, 102126)	0.04320393518322164
  (0, 102073)	0.13601960923329875
  (0, 101925)	0.07130835583574051
  (0, 100627)	0.03944940350821724
  (0, 100055)	0.027120025057598224
  :	:
  (20799, 7627)	0.010633825879980713
  (20799, 7297)	0.028162791439370075
  (20799, 7001)	0.039589284861090264
  (20799, 6963)	0.0253

In [46]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [47]:
X.shape

(20800, 112643)

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2, stratify=Y)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((16640, 112643), (4160, 112643), (16640,), (4160,))

Model Training

In [64]:
model = LogisticRegression()

In [65]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation Model

In [66]:
X_train_pred = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_pred, Y_train)
training_data_accuracy

0.9801081730769231

Checking Accuracy on Test Data

In [67]:
X_test_pred = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, Y_test)
test_data_accuracy

0.9538461538461539